.. index:: Equations; Cahn-Hilliard

# Cahn-Hilliard
In this script we show how to use the $C^1$ non-conforming virtual
element space to solve the Cahn-Hilliard equation. We use a fully implicit
scheme here. You can find more details in <cite data-cite="VEM"></cite>

In [1]:
try:
    import dune.vem
except:
    print("This example needs 'dune.vem' - skipping")
    import sys
    sys.exit(0)
from matplotlib import pyplot
import random
from dune.grid import cartesianDomain, gridFunction
import dune.fem
from dune.fem.plotting import plotPointData as plot
from dune.fem.function import discreteFunction

from ufl import *
import dune.ufl

dune.fem.threading.use = 4

Grid and space construction - we use a cube grid here:

In [2]:
order        = 3
polyGrid = dune.vem.polyGrid( cartesianDomain([0,0],[1,1],[30,30]), cubes=True)
testSpaces = [ [0], [order-3,order-2], [order-4] ]
# testSpaces   = [ [0,0],[order-4,order-3], [order-4] ]
space = dune.vem.vemSpace(polyGrid, order=order, testSpaces=testSpaces)

To define the mathematical model, let $\psi\colon{\mathbb R} \rightarrow
\mathbb{R}$ be defined as
$\psi(x) = \frac{(1-x^2)^2}{4}$ and let $\phi(x) = \psi(x)^{\prime}$.
The strong form for the solution
$u\colon \Omega \times [0,T] \rightarrow {\mathbb R}$
is given by
\begin{align*}
\partial_t u  - \Delta (\phi(u)-\epsilon^2 \Delta u) = 0
\quad &\text{in} \ \Omega \times [0,T] ,\\
u(\cdot,0) = u_0(\cdot)  \quad &\text{in} \ \Omega,\\
\partial_n u = \partial_n \big( \phi(u) - \epsilon^2\Delta u \big) = 0
\quad &\text{on} \ \partial \Omega \times [0,T].
\end{align*}

We use a backward Euler discretization in time and will fix the constant
further down:

In [3]:
t     = dune.ufl.Constant(0,"time")
tau   = dune.ufl.Constant(0,"dt")
eps   = dune.ufl.Constant(0,"eps")
df_n  = discreteFunction(space, name="oldSolution") # previous solution
x     = SpatialCoordinate(space)
u     = TrialFunction(space)
v     = TestFunction(space)

H     = lambda v: grad(grad(v))
laplace = lambda v: H(v)[0,0]+H(v)[1,1]
a     = lambda u,v: inner(H(u),H(v))
b     = lambda u,v: inner( grad(u),grad(v) )
W     = lambda v: 1/4*(v**2-1)**2
dW    = lambda v: (v**2-1)*v

equation = ( u*v + tau*eps*eps*a(u,v) + tau*b(dW(u),v) ) * dx == df_n*v * dx

dbc = [dune.ufl.DirichletBC(space, 0, i+1) for i in range(4)]

# energy
Eint  = lambda v: eps*eps/2*inner(grad(v),grad(v))+W(v)

Next we construct the scheme providing some suitable expressions to stabilize the method

In [4]:
biLaplaceCoeff = eps*eps*tau
diffCoeff      = 2*tau
massCoeff      = 1
scheme = dune.vem.vemScheme(
                       [equation, *dbc],
                       solver=("suitesparse","umfpack"),
                       hessStabilization=biLaplaceCoeff,
                       gradStabilization=diffCoeff,
                       massStabilization=massCoeff,
                       boundary="derivative") # only fix the normal derivative = 0

To avoid problems with over- and undershoots we project the initial
conditions into a linear lagrange space before interpolating into the VEM
space:

In [5]:
def initial(x):
    h = 0.01
    g0 = lambda x,x0,T: conditional(x-x0<-T/2,0,conditional(x-x0>T/2,0,sin(2*pi/T*(x-x0))**3))
    G  = lambda x,y,x0,y0,T: g0(x,x0,T)*g0(y,y0,T)
    return 0.5*G(x[0],x[1],0.5,0.5,50*h)
initial = dune.fem.space.lagrange(polyGrid,order=1).interpolate(initial(x),name="initial")
df = space.interpolate(initial, name="solution")

Finally the time loop - for the final coarsening phase (time greater than 0.8)
we increase the time step a bit:

In [6]:
t.value = 0
eps.value = 0.05

fig = pyplot.figure(figsize=(30,30))
count = 0
pos = 1
while t.value < 2.15:
    if t.value < 0.6:
        tau.value = 1e-02
    else:
        tau.value = 5e-02
    df_n.assign(df)
    info = scheme.solve(target=df)
    t.value += tau
    count += 1
    if count % 10 == 0:
        df.plot(figure=(fig,330+pos),colorbar=None,clim=[-1,1])
        energy = dune.fem.integrate(Eint(df),order=3)
        print("[",pos,"]",t.value,tau.value,energy,info,flush=True)
        pos += 1

[ 1 ] 0.09999999999999999 0.01 0.2109105656204094 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.686184262, 0.425560663, 0.26062359900000004]}


[ 2 ] 0.20000000000000004 0.01 0.15979702768268475 {'converged': True, 'iterations': 3, 'linear_iterations': 3, 'timing': [1.03139613, 0.638997412, 0.39239871800000004]}


[ 3 ] 0.3000000000000001 0.01 0.14625128437425206 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.686196089, 0.42493244100000005, 0.2612636479999999]}


[ 4 ] 0.4000000000000002 0.01 0.13650640507974696 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.689934701, 0.42698471800000004, 0.26294998299999994]}


[ 5 ] 0.5000000000000002 0.01 0.12678192004910677 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.699805569, 0.44206644800000006, 0.25773912099999996]}


[ 6 ] 0.6000000000000003 0.01 0.11686029348491062 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.696169825, 0.42634904900000004, 0.269820776]}


[ 7 ] 1.1000000000000008 0.05 0.08265470257362988 {'converged': True, 'iterations': 2, 'linear_iterations': 2, 'timing': [0.691568095, 0.427225121, 0.26434297400000006]}


[ 8 ] 1.6000000000000012 0.05 0.08265484990090012 {'converged': True, 'iterations': 1, 'linear_iterations': 1, 'timing': [0.358614033, 0.203612666, 0.155001367]}


[ 9 ] 2.100000000000001 0.05 0.08265485993702593 {'converged': True, 'iterations': 1, 'linear_iterations': 1, 'timing': [0.360719924, 0.20409924000000002, 0.156620684]}


<Figure size 1500x1500 with 0 Axes>